In [1]:
#!g2.1
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
#!g2.1
from datasets import load_dataset
raw_datasets = load_dataset("alexcadillon/SemEval2014Task4", 'restaurants')
raw_datasets

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 115.31it/s]
Generating trial split: 100 examples [00:00, 1025.54 examples/s]
Generating train split: 3041 examples [00:00, 10711.67 examples/s]
Generating test split: 800 examples [00:00, 4845.66 examples/s]


DatasetDict({
    trial: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories'],
        num_rows: 100
    })
    train: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories'],
        num_rows: 3041
    })
    test: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories'],
        num_rows: 800
    })
})

In [3]:
#!g2.1
categories = []
for i in raw_datasets['train']['aspectCategories']:
    if i[0]['category'] not in categories:
        categories.append(i[0]['category'])
categories    

['service', 'food', 'anecdotes/miscellaneous', 'ambience', 'price']

In [4]:
#!g2.1
labels = ['service', 'food', 'anecdotes/miscellaneous', 'ambience', 'price']


In [5]:
#!g2.1
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
label2id

{'service': 0,
 'food': 1,
 'anecdotes/miscellaneous': 2,
 'ambience': 3,
 'price': 4}

In [6]:
#!g2.1
sample = raw_datasets['train'][5]
sample

{'sentenceId': '2846',
 'text': "Not only was the food outstanding, but the little 'perks' were great.",
 'aspectTerms': [{'term': 'food',
   'polarity': 'positive',
   'from': '17',
   'to': '21'},
  {'term': 'perks', 'polarity': 'positive', 'from': '51', 'to': '56'}],
 'aspectCategories': [{'category': 'food', 'polarity': 'positive'},
  {'category': 'service', 'polarity': 'positive'}]}

In [12]:
#!g2.1
def combo_label(example):   
    labels_combo = []
    for item in example['aspectCategories']:
        labels_combo.append(f"{item['category']}")
    example['labels_combo'] = labels_combo
    return example

In [13]:
#!g2.1
dataset = raw_datasets.map(combo_label)
dataset

Map: 100%|██████████| 800/800 [00:00<00:00, 11480.04 examples/s]


DatasetDict({
    trial: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories', 'labels_combo'],
        num_rows: 100
    })
    train: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories', 'labels_combo'],
        num_rows: 3041
    })
    test: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspectCategories', 'labels_combo'],
        num_rows: 800
    })
})

In [14]:
#!g2.1
sample = dataset['train'][0]
sample

{'sentenceId': '3121',
 'text': 'But the staff was so horrible to us.',
 'aspectTerms': [{'term': 'staff',
   'polarity': 'negative',
   'from': '8',
   'to': '13'}],
 'aspectCategories': [{'category': 'service', 'polarity': 'negative'}],
 'labels_combo': ['service']}

In [15]:
#!g2.1
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("albert-xxlarge-v2")
label_counts = len(labels)

def preprocess_data(example):
  # take a batch of texts
  text = example["text"]
  # encode them
  encoding = tokenizer(text, add_special_tokens=True)
  # add labels
  label_ids = [0 for i in range(label_counts)]
  for item in example['labels_combo']:
            new = [1.0 if l == labels.index(item) else 0.0 for l in range(label_counts)]
            label_ids = [x+y for x,y in zip(label_ids, new)]
            new = [0 for i in range(label_counts)]

  encoding["labels"] = label_ids

  return encoding

In [16]:
#!g2.1
preprocess_data(sample)

{'input_ids': [2, 47, 14, 1138, 23, 86, 9244, 20, 182, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1.0, 0.0, 0.0, 0.0, 0.0]}

In [17]:
#!g2.1
tokenized_dataset = dataset.map(preprocess_data, remove_columns=dataset['train'].column_names)
tokenized_dataset

Map: 100%|██████████| 800/800 [00:00<00:00, 3883.64 examples/s]


DatasetDict({
    trial: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3041
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
})

In [18]:
#!g2.1
raw_datasets['train'][15]

{'sentenceId': '3359',
 'text': 'The pizza is the best if you like thin crusted pizza.',
 'aspectTerms': [{'term': 'pizza',
   'polarity': 'positive',
   'from': '4',
   'to': '9'},
  {'term': 'thin crusted pizza',
   'polarity': 'neutral',
   'from': '34',
   'to': '52'}],
 'aspectCategories': [{'category': 'food', 'polarity': 'positive'}]}

In [19]:
#!g2.1
example = tokenized_dataset['train'][15]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [20]:
#!g2.1
tokenizer.decode(example['input_ids'])

2024-04-09 20:38:06.639337: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'[CLS] the pizza is the best if you like thin crusted pizza.[SEP]'

In [21]:
#!g2.1
example['labels']

[0.0, 1.0, 0.0, 0.0, 0.0]

In [22]:
#!g2.1
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['food']

In [23]:
#!g2.1
tokenized_dataset.set_format("torch")

In [24]:
#!g2.1
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("albert-xxlarge-v2", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors: 100%|██████████| 893M/893M [00:13<00:00, 65.0MB/s] 
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-xxlarge-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
#!g2.1
batch_size = 8

In [26]:
#!g2.1
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = True)

In [27]:
#!g2.1
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "no",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    #push_to_hub=True,
)

In [28]:
#!g2.1
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'macro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [29]:
#!g2.1
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator = data_collator
)

trainer.train()

 99%|█████████▉| 99/100 [00:11<00:00,  6.67it/s]
                                                  [A
100%|██████████| 100/100 [00:12<00:00,  6.65it/s]
                                                 

{'eval_loss': 0.14634805917739868, 'eval_f1': 0.8792216546313046, 'eval_roc_auc': 0.908165687613814, 'eval_accuracy': 0.81375, 'eval_runtime': 12.1404, 'eval_samples_per_second': 65.895, 'eval_steps_per_second': 8.237, 'epoch': 1.0}


 26%|██▌       | 500/1905 [03:26<09:37,  2.43it/s]  

{'loss': 0.2673, 'learning_rate': 1.4750656167979002e-05, 'epoch': 1.31}


100%|██████████| 100/100 [00:12<00:00,  6.64it/s]
                                                  
 40%|████      | 762/1905 [05:17<06:04,  3.14it/s][A
                                                 

...

100%|██████████| 100/100 [00:12<00:00,  6.61it/s]
                                                   
 80%|████████  | 1524/1905 [10:35<02:41,  2.36it/s]A
                                                 

{'eval_loss': 0.1539347618818283, 'eval_f1': 0.9332828420197249, 'eval_roc_auc': 0.9543375297253321, 'eval_accuracy': 0.86875, 'eval_runtime': 12.1395, 'eval_samples_per_second': 65.9, 'eval_steps_per_second': 8.238, 'epoch': 4.0}


100%|██████████| 100/100 [00:11<00:00,  6.63it/s]
                                                   
100%|██████████| 1905/1905 [13:14<00:00,  2.40it/s]A

{'eval_loss': 0.15995360910892487, 'eval_f1': 0.9344935829831197, 'eval_roc_auc': 0.953860251743541, 'eval_accuracy': 0.8725, 'eval_runtime': 12.1211, 'eval_samples_per_second': 66.001, 'eval_steps_per_second': 8.25, 'epoch': 5.0}
{'train_runtime': 794.8816, 'train_samples_per_second': 19.129, 'train_steps_per_second': 2.397, 'train_loss': 0.10925025113924282, 'epoch': 5.0}


TrainOutput(global_step=1905, training_loss=0.10925025113924282, metrics={'train_runtime': 794.8816, 'train_samples_per_second': 19.129, 'train_steps_per_second': 2.397, 'train_loss': 0.10925025113924282, 'epoch': 5.0})